#  Capstone Project Assigment


In [ ]:
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from bs4 import BeautifulSoup

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1.Start by creating a new Notebook for this assignment.

2.Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)

3.Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

4.More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

5.If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

6.Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making

In [ ]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    tds = tr.find_all('td')
    for i in tds:
        rows.append(i)
        
lst=[]
bor=""
postal=""
for row in rows:
    neigh=""
    if row.span.text[0:13] != 'Not assigned':
        try:
            vals=row.span.find_all('a')
            for v in range(len(vals)):
                if v == 0:
                    bor=row.span.find_all('a')[v].text
                else:
                    if neigh == "":
                        neigh=row.span.find_all('a')[v].text
                    else:
                        neigh=neigh+","+row.span.find_all('a')[v].text+","+neigh
            postal=row.b.text
            if neigh=="":
                lst.append([postal, bor, bor])
            else:
                lst.append([postal, bor, neigh])                
        except Exception as e:
            e

7. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [ ]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)

8. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [ ]:
df.shape

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [ ]:
dfgeo = pd.read_csv("Geospatial_Coordinates.csv")
dfgeo.head()

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:


Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code

In [ ]:
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df2 = pd.merge(df, dfgeo, on="PostalCode", how='left')

In [ ]:
df2

# Create a new dataframe for only boroughs that contain the word Toronto

In [ ]:
Toronto=df2[df2['Borough'].str.contains('Toronto')]
Toronto

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

1.to add enough Markdown cells to explain what you decided to do and to report any observations you make.

2.to generate maps to visualize your neighborhoods and how they cluster together.

In [ ]:
address = 'Toronto'
geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(Toronto['Latitude'], Toronto['Longitude'], 
                                           Toronto['Borough'], Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_map)  
    
Toronto_map